In [2]:
import pathlib

import pandas as pd

In [33]:
results_files = pathlib.Path("./data/").glob("results*")
results = []
for f in results_files:
    df = pd.read_csv(f)
    parts = str(f).replace(".csv", "").split("_")
    df['num_agents'] = int(parts[1])
    df['num_items'] = int(parts[2])
    df['num_weeks'] = int(parts[3])
    results.append(df)
results = pd.concat(results)

In [50]:
def parse_results(s):
    parts = s.split(",")
    stat = float(parts[0].split("=")[-1])
    pval = float(parts[1].split("=")[-1].replace(")", ""))
    return [stat, pval]

In [52]:
results['time_split'] = results['time_ks'].apply(parse_results)
results['churn_split'] = results['churn_ks'].apply(parse_results)

In [58]:
time = pd.DataFrame(results['time_split'].tolist(), columns=["time_stat", "time_pval"])
churn = pd.DataFrame(results['churn_split'].tolist(), columns=["churn_stat", "churn_pval"])
results = pd.concat([results.drop(columns=['time_ks', 'churn_ks', 'time_split', 'churn_split']), time, churn], axis=1)

In [60]:
results.sort_values(by=["mean_jsd", "time_stat", "churn_stat"], ascending=True)

,pop,pl,mean_jsd,num_agents,num_items,num_weeks,time_stat,time_pval,churn_stat,churn_pval
15,0.75,1.00,0.156501,1000000,116,36,0.384615,0.299920,0.285714,1.150772e-01
12,0.50,1.50,0.161959,1000000,116,36,0.230769,0.897806,0.514286,1.404438e-04
14,0.50,2.00,0.172565,1000000,116,36,0.230769,0.897806,0.428571,2.885997e-03
17,0.75,1.50,0.182910,1000000,116,36,0.307692,0.588196,0.371429,1.530453e-02
0,0.00,1.00,0.187370,1000000,116,36,0.250000,0.716412,0.571429,1.286219e-05
10,0.50,1.00,0.195116,1000000,116,36,0.307692,0.588196,0.571429,1.286219e-05
11,0.50,1.25,0.195247,1000000,116,36,0.357143,0.343320,0.628571,8.461968e-07
5,0.25,1.00,0.197050,1000000,116,36,0.266667,0.678138,0.685714,3.857589e-08
19,0.75,2.00,0.198568,1000000,116,36,0.307692,0.588196,0.428571,2.885997e-03
7,0.25,1.50,0.199240,1000000,116,36,0.307692,0.588196,0.685714,3.857589e-08
